# Data Clustering

<a rel="license" href="https://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons Licence" style="width=50" src="https://licensebuttons.net/l/by-nc-sa/4.0/88x31.png" title='This work is licensed under a Creative Commons Attribution 4.0 International License.' align="right"/></a>

**Authors**: Dr Matteo Degiacomi (matteo.t.degiacomi@durham.ac.uk) and Dr Antonia Mey (antonia.mey@ed.ac.uk)

Content is partially adapted from the [Software Carpentries Machine learning lesson](https://carpentries-incubator.github.io/machine-learning-novice-sklearn/index.html) and [here](https://github.com/christianversloot/machine-learning-articles/blob/main/performing-dbscan-clustering-with-python-and-scikit-learn.md).

**Learning Objectives:**
- Identify clusters in data using *k-means* clustering, *DBSCAN* and *spectral clustering*. 
- See the limitations of k-means when clusters overlap.
- Use other clustering methods to overcome the limitations of k-means.

**Jupyter cheat sheet**
- to run the currently highlighted cell, hold <kbd>&#x21E7; Shift</kbd> and press <kbd>&#x23ce; Enter</kbd>;
- to get help for a specific function, place the cursor within the function's brackets, hold <kbd>&#x21E7; Shift</kbd>, and press <kbd>&#x21E5; Tab</kbd>;

<div class="alert alert-info"><b> Remember: variables persist between cells</b> 
    
Be aware that it is the order of execution of cells that is important in a Jupyter notebook, not the <em>order</em> in which they appear. Python will remember <em>all</em> the code that was run previously, including any variables you have defined, irrespective of the order in the notebook. Therefore if you define variables lower down the notebook and then (re)run cells further up, those defined further down will still be present. </div> 

## Table of Contents
[0.   Google Colab Setup](#setup)      
[1.   Introduction](#intro)   
[2.   K-means clustering](#kmeans)   
[3.   DBSCAN: a density-based clustering scheme](#dbscan)   
[4.   Dihedral space of alanine dipeptide](#dihedrals)    
[5.   Conclusion](#conclusion)  

## 0. Google Colab setup
<a id='setup'></a>
<div class="alert alert-warning">
<b>Attention:</b> Please only run the following cells if you are using Colab! These cells install necessary packages and download data.</div>

In [ ]:
!if [ -n "$COLAB_RELEASE_TAG" ]; then pip install condacolab; fi
import condacolab
condacolab.install()

import condacolab
!mamba install -c conda-forge mdanalysis mdanalysistests mdanalysisdata nglview scikit-learn ipywidgets=7.6.0

In [ ]:
# enable third party jupyter widgets
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
!if [ -n "$COLAB_RELEASE_TAG" ]; then git clone https://github.com/CCPBioSim/CCP5_Simulation_of_BioMolecules; fi
import os
os.chdir(f"CCP5_Simulation_of_BioMolecules{os.sep}7_Analysis_clustering")

## 1. Introduction
<a id='intro'></a>

Clustering is the grouping of data points which are similar to each other. It can be a powerful technique for identifying patterns in data. Clustering is known as an *unsupervised learning* technique, whereby no example data needs to be provided for this task to be carried out. Application of clustering are:
- Looking for trends in data
- Data compression, all data clustering around a point can be reduced to just that point (e.g., reducing colour depth of an image)
- Pattern recognition.

In this tutorial, we will look at common clustering techniques, k-Means, DBASCAN, and Spectral Clustering, and we will apply them first to toy data, and then on the molecular dynamics simulation on alanine dipeptide.

## 2. K-means clustering
<a id='kmeans'></a>

K-means is an algorithm that aggregates data into clusters by manipulating the position of cluster centres (centroids). Its goal is to search for the ideal position of centrods such that the average distance between each data point and its closest centroid is minimized. The algorithm needs to be told **how many clusters** to define, but a common technique is to try different numbers of clusters and combine it with other tests to decide on the number.

To perform a [K-means](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) clustering with Scikit-learn, we first need to import the `sklearn.cluster` module.

In [ ]:
import sklearn.cluster as skl_cluster
import numpy as np

### 2.1. Creating the data

For this example, we will use scikit-learn’s built in random data blob generator instead of using an external dataset. For this we will also need the `sklearn.datasets.samples_generator` module.

In [ ]:
import sklearn.datasets as skl_datasets

Now let’s create some random blobs using the `make_blobs` function. The `n_samples` argument sets how many points we want to use in all of our blobs. `cluster_std` sets the standard deviation of the points, the smaller this value the closer together they will be. `centers` sets the desired number of clusters. `random_state` is the initial state of the random number generator, by specifying its value we will get the same results every time we run the program. If we do not specify a random state, then we will get different points every time we run. This function returns two things: an array of data points and a list of which cluster each point belongs to.

In [ ]:
data, cluster_id = skl_datasets.make_blobs(n_samples=400, cluster_std=0.75, centers=4, random_state=1)

### 2.2. Plotting and clustering the data

We have now created the data, let's have a look at what `make_blobs` gave us!

In [ ]:
np.shape(data)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(data[:, 0], data[:, 1], s=5, linewidth=0)

Let's identify the clusters using K-means. First, we need to initialise the `KMeans` module. As parameters, we tell it how many clusters to look for (`n_clusters=4`), and how many times to repeat the clustering (`n_init=10`, the attempt yielding the smallest average distance of datapoints from the closest centroid will be returned). The latter parameter is especially useful for sparse data, where the performance of `KMeans` is more affected by poor initial conditions. Next, we define what data needs to be clustered via the `fit` function. Finally, we run the predict function to find the clusters.

In [ ]:
# 1. Initialise Kmeans
Kmean = skl_cluster.KMeans(n_clusters=4, n_init=10)
# 2. Fit the data
Kmean.fit(data)
# 3. Predict the clusters
clusters = Kmean.predict(data)

The data can now be plotted to show all the points we randomly generated. To clarify to which cluster each data point has been assigned to, we can define their colour (the `c` parameter) using the <code>clusters</code> list of integers returned by the predict function. Data points belonging to the same cluster will be associated to the same integer number in this list. The K-means algorithm also lets us know where the centre of each cluster is located. Cluster centres are stored as a list called `cluster_centers_` inside the `Kmean` object. Let’s go ahead and plot the points from the clusters, colouring them by the output from the K-means algorithm, and also plotting the centres of each cluster as a red triangle.

In [ ]:
plt.scatter(data[:, 0], data[:, 1], s=5, linewidth=0, c=clusters)
for cluster_x, cluster_y in Kmean.cluster_centers_:
    plt.scatter(cluster_x, cluster_y, s=100, c='r', marker='^')
plt.show()

<div class="alert alert-info">
<b>Working in multiple dimensions.</b>
Data manipulated in a machine learning context is ofter high-dimensional. Although the example presented her shows the clustering of 2-dimensional data, the k-means algorithm can operate in an arbitrarily high number of dimensions. It is however difficult to show this visually once we get beyond 3 dimensions.
</div>


### 2.3. Task section
Please work in small groups for the next 15 minutes on the tasks below

<div class="alert alert-success">
<b>Task 1.</b> Discuss: What are the limitations and advantages of K-Means? </div>


<details>
<summary> <mark> Solution: Suggested limitations and advantages</mark> </summary>

Limitations:
- Requires number of clusters to be known in advance,
- Struggles when clusters have irregular shapes,
- Will always produce an answer finding the required number of clusters even if the data features a different number of clusters, or no cluster at all,
- Requires linear cluster boundaries.

Advantages:
- Simple algorithm, fast to compute. A good choice as the first thing to try when attempting to cluster data,
- Suitable for large datasets due to its low memory and computing requirements.

</details>

<div class="alert alert-success">
<b>Task 2.</b> K-means with overlapping clusters. Adjust the program below to increase the standard deviation of the blobs (the <code>cluster_std</code> parameter to <code>make_blobs</code>) and increase the number of samples (<code>n_samples</code>) to 4000. Visually, you should start to see the blobs start overlapping. Do the clusters that are identified make sense?
    
```Python
data, cluster_id = skl_datasets.make_blobs(n_samples=400, cluster_std=0.75, centers=4, random_state=1)

Kmean = skl_cluster.KMeans(n_clusters=4)
Kmean.fit(data)
clusters = Kmean.predict(data)

plt.scatter(data[:, 0], data[:, 1], s=5, linewidth=0, c=clusters)
for cluster_x, cluster_y in Kmean.cluster_centers_:
    plt.scatter(cluster_x, cluster_y, s=100, c='r', marker='^')

```


In [ ]:
### Your solution here! ###


<details>
<summary> <mark> Solution</mark> </summary>

```Python
data, cluster_id = skl_datasets.make_blobs(n_samples=4000, cluster_std=3.0, centers=4, random_state=1)

Kmean = skl_cluster.KMeans(n_clusters=4)
Kmean.fit(data)
clusters = Kmean.predict(data)

plt.scatter(data[:, 0], data[:, 1], s=5, linewidth=0, c=clusters)
for cluster_x, cluster_y in Kmean.cluster_centers_:
    plt.scatter(cluster_x, cluster_y, s=100, c='r', marker='^')
```

</details>

<div class="alert alert-success">
<b>Task 3.</b> How many clusters should we look for?
As K-Means requires us to specify the number of clusters to expect a common strategy to get around this is to vary the number of clusters we are looking for. Modify the program to loop through searching for between 2 and 10 clusters. Which (if any) of the results look more sensible? What criteria might you use to select the best one?
</div>

In [ ]:
### Your solution here! ###


<details>
<summary> <mark> Solution</mark> </summary>

```Python
for cluster_count in range(2,11):
    Kmean = skl_cluster.KMeans(n_clusters=cluster_count)
    Kmean.fit(data)
    clusters = Kmean.predict(data)
    plt.scatter(data[:, 0], data[:, 1], s=5, linewidth=0,c=clusters)
    for cluster_x, cluster_y in Kmean.cluster_centers_:
        plt.scatter(cluster_x, cluster_y, s=100, c='r', marker='^')
        # give the graph a title with the number of clusters
        plt.title(str(cluster_count)+" Clusters")
    plt.show()
```
None of these look very sensible clusterings because all the points really form one large cluster. We might look at a measure of similarity of the cluster to test if its really multiple clusters. A simple standard deviation or interquartile range might be a good starting point.

</details>

## 3. DBSCAN: a density based clustering scheme
<a id='dbscan'></a>

### 3.1. Generating and plotting the data

In some cases, data might be arranged according to non-spherical clusters. An (extreme) example of such a situation, is that of data points organized in two concentric rings. Scikit-learn has an inbuilt version of generating such data, similar to the `make_blobs` function, it is called `make_circles`.

In [ ]:
import sklearn.datasets as skl_data
circles, circles_clusters = skl_data.make_circles(n_samples=400, noise=.01, random_state=0)

Let's plot the data to see what it looks like

In [ ]:
plt.scatter(circles[:, 0], circles[:, 1], s=5, linewidth=0)

### 3.2. Clustering the data

What happens if we use k-means to cluster this data?

In [ ]:
Kmean = skl_cluster.KMeans(n_clusters=2)
Kmean.fit(circles)
clusters = Kmean.predict(circles)

plt.scatter(circles[:, 0], circles[:, 1], s=5, linewidth=0, c=clusters)
for cluster_x, cluster_y in Kmean.cluster_centers_:
    plt.scatter(cluster_x, cluster_y, s=100, c='r', marker='^')

Oh dear... this is not ideal! Scikit-learn offers other clustering algorithm. Let's try [DBSCAN](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) (Density-Based Spatial Clustering of Applications with Noise).

In [ ]:
db = skl_cluster.DBSCAN(eps=0.1)
db.fit(circles)
clusters = db.labels_.astype(int)
no_clusters = len(np.unique(clusters) )
no_noise = np.sum(np.array(clusters) == -1, axis=0)

print(f'Estimated no. of clusters: {no_clusters}')
print(f'Estimated no. of noise points: {no_noise}')

In [ ]:
colors = list(map(lambda x: '#3b4cc0' if x == 1 else '#b40426', clusters))
plt.scatter(circles[:,0], circles[:,1], c=colors, marker="o", picker=True)

DBSCAN produced a much better clustering. This algorithm iteratively aggregates points to clusters according to a distance criterion `eps`. A point will be added to a cluster if it is at a distance smaller than `eps` from any point already part of it. As a result, the clusters discovered via DBSCAN can have any shape, and their number does not need to be known in advance. An interesting feature of DBSCAN is that it can be also used to identify data outliers (i.e. points that are not assigned to any cluster).

### 3.3. Task section
Please work in small groups for the next 15 minutes on the tasks below

<div class="alert alert-success">
<b>Task 4.</b> What happens when you change the epsilon value of DBSCAN? </div>
For the two rings it is quite obvious that you want two clusters to describe this. DBSCAN can be used to identify different numbers of clusters. Play around with the epsilon value to figure out at which point DBSCAN may fail in classifying the two rings. 

In [ ]:
### Your solution here! ###


<details>
<summary> <mark> Solution</mark> </summary>
    
```Python
epsilon = np.linspace(0.05,0.3,10)
for eps in epsilon:
    db = skl_cluster.DBSCAN(eps=eps)
    db.fit(circles)
    clusters = db.labels_.astype(int)
    no_clusters = len(np.unique(clusters) )
    no_noise = np.sum(np.array(clusters) == -1, axis=0)
    print(f'=========== Current epsilon value is: {eps:.2f} ==========')
    print(f'Estimated no. of clusters: {no_clusters}')
    print(f'Estimated no. of noise points: {no_noise}')
```
</details>

<div class="alert alert-success">
<b>Task 5.</b> Can spectral clustering also recognise the two rings? </div>

You have seen that k-means is bad at distinguishing the two rings. DBSCAN did manage to do this with a well chosen epsilon value. Another method called Spectral Clustering can also be used to cluster data. Look at the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.SpectralClustering.html) and see if you can get the spectral clustering method to work for the ring dataset. 

In [ ]:
### Your solution here! ###


<details>
<summary> <mark> Solution</mark> </summary>

The code for calculating the SpectralClustering is very similar to the k-means clustering, instead of using the `sklearn.cluster.KMeans` class we use the `sklearn.cluster.SpectralClustering` class. The `SpectralClustering` class combines the `fit` and `predict` functions into a single function called `fit_predict`. This is how the solution for clustering the dataset using Spectral clustering looks like:

```Python
import sklearn.cluster as skl_cluster
import sklearn.datasets as skl_data

circles, circles_clusters = skl_data.make_circles(n_samples=1000, noise=.01, random_state=0)

# cluster with spectral clustering
model = skl_cluster.SpectralClustering(n_clusters=2, affinity='nearest_neighbors', assign_labels='kmeans')
labels = model.fit_predict(circles)
#colors = list(map(lambda x: '#3b7cc0' if x == 1 else '#111426', labels))
plt.scatter(circles[:, 0], circles[:, 1], s=15, linewidth=0, c=labels)
plt.show()
```
    
</details>

<div class="alert alert-success">
<b>Task 6 (Optional).</b> Performance of clustering algorithms. 

Combine the code from the k-means clustering we have shown above for the two circles and your solution to the last task to using the spectral clustering. Using the `time` module, time how long both clustering algorithms take to run. To get the start time, get something like `start_time = time.time()`. At the end of the clustering snipptet part for each clustering method get `finish_time` in the same way as `start_time`. Subtracting each of the time components you can figure out how long the spectral and k-means clustering takes. Compare how long both parts take to run generating 4,000 samples and testing them for between 2 and 10 clusters. How much did your run times differ? How much do they differ if you increase the number of samples to 8,000? How long do you think it would take to compute 800,000 samples (estimate this, it might take a while to run for real)?</div>

<details>
<summary> <mark> Solution</mark> </summary>

Runtime is typically ~4 seconds (depending on the computer used)

```Python
import matplotlib.pyplot as plt
import sklearn.cluster as skl_cluster
from sklearn.datasets import make_blobs 
import time

start_time = time.time()
data, cluster_id = make_blobs(n_samples=4000, cluster_std=3,
                                       centers=4, random_state=1)

for cluster_count in range(2,11):
    Kmean = skl_cluster.KMeans(n_clusters=cluster_count)
    Kmean.fit(data)
    clusters = Kmean.predict(data)

    plt.scatter(data[:, 0], data[:, 1], s=15, linewidth=0, c=clusters)
    plt.title(str(cluster_count)+" Clusters")

plt.show()

end_time = time.time()
print("Elapsed time = ", end_time-start_time, "seconds")
```
<br>
Spectral version, runtime around 9 seconds (your computer might be faster/slower)
<br>
    
```Python
import matplotlib.pyplot as plt
import sklearn.cluster as skl_cluster
from sklearn.datasets import make_blobs 
import time

start_time = time.time()
data, cluster_id = make_blobs(n_samples=4000, cluster_std=3,
                                       centers=4, random_state=1)

for cluster_count in range(2,11):
    model = skl_cluster.SpectralClustering(n_clusters=cluster_count,
                                       affinity='nearest_neighbors',
                                       assign_labels='kmeans')
    labels = model.fit_predict(data)
    
    plt.scatter(data[:, 0], data[:, 1], s=15, linewidth=0, c=labels)
    plt.title(str(cluster_count)+" Clusters")
plt.show()
end_time = time.time()
print("Elapsed time = ", end_time-start_time, "seconds")
    
```
    
When the number of points increases to 8000 the runtimes are 24 seconds for the spectral version and 5.6 seconds for kmeans. The runtime numbers will differ depending on the speed of your computer, but the relative different should be similar. For 4000 points kmeans took 4 seconds, spectral 9 seconds, 2.25 fold difference. For 8000 points kmeans took 5.6 seconds, spectral took 24 seconds. 4.28 fold difference. Kmeans 1.4 times slower for double the data, spectral 2.6 times slower. The realative difference is diverging. Its double by doubling the amount of data. If we use 100 times more data we might expect a 100 fold divergence in execution times. Kmeans might take a few minutes, spectral will take hours.
    
</details>

## 4. Dihedral space of alanine dipeptide
<a id='dihedrals'></a>

### 4.1. Loading and plotting the data

Alanine dipeptide (ADP) is a commonly used toy model in molecular simulations. Let's start by importing some useful packages.

In [ ]:
import nglview as nv
import warnings
import os
import MDAnalysis as mda
import numpy as np
warnings.filterwarnings("ignore")

Let us load ADP into `nglview` to look at the molecule in three dimensions.

In [ ]:
adp = mda.Universe(os.path.join('..', 'data', 'alanine-dipeptide-nowater.pdb' ))
w = nv.show_mdanalysis(adp)
w

We will now load information on the backbone dihedral angles $\phi$ and $\psi$ of three 250 ns-long simulations of alanine dipeptide.

In [ ]:
temp = np.load(os.path.join('..', 'data', 'alanine-dipeptide-3x250ns-backbone-dihedrals.npz'), mmap_mode='r')
dihedral = []
for k in temp.files:
    dihedral.append(temp[k])
dihedral_data = np.concatenate(dihedral)

`dihedral_data` contains the $\phi$ and $\psi$ dihedral angles. Let's take a look at their distribution. 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.5)
sns.set_style('ticks')

plt.plot(dihedral_data[:,0], dihedral_data[:,1], lw=0, marker='o', ms=0.5)
plt.xlabel(r'$\phi$ in [rad]')
plt.ylabel(r'$\psi$ in [rad]')

### 4.2. Task section

<div class="alert alert-success">
<b>Task 7.</b> What would be a better way to represent this data than a scatter plot? </div>


<details>
<summary> <mark> Solution</mark> </summary>
The scatter plot we have just produced, called the Ramachandran plot, is often visualised as a probability density.
</details>

<div class="alert alert-success">
<b>Task 8.</b> Try using k-means clustering for the dataset. What do you think is the right number of cluster centers? Can you spot a problem with the clustering of this dataset?
</div>

In [ ]:
### Your solution here! ###


<details>
<summary> <mark> Solution</mark> </summary>

```Python
# cluster the data
Kmean = skl_cluster.KMeans(n_clusters=100)
Kmean.fit(dihedral_data)
clusters = Kmean.predict(dihedral_data)
    
# plot the ata
plt.scatter(dihedral_data[:, 0], dihedral_data[:, 1], s=5, linewidth=0)
for cluster_x, cluster_y in Kmean.cluster_centers_:
    plt.scatter(cluster_x, cluster_y, s=100, c='r', marker='^')
plt.xlabel(r'$\phi$ in [rad]')
plt.ylabel(r'$\psi$ in [rad]')
```
<br>
It is hard to tell the exact number of clusters, but 2-4 seems like a suitable answer. A problem in this data, is that dihedral angles are periodic.
    
</details>

<div class="alert alert-success">
<b>Task 9.</b>  Try using DBSCAN to cluster this dataset.
</div>


In [ ]:
### Your solution here! ###


<details>
<summary> <mark> Solution</mark> </summary>

```Python
# cluster the data
db = skl_cluster.DBSCAN(eps=0.2)
db.fit(dihedral_data[::10])
clusters = db.labels_.astype(int)
no_clusters = len(np.unique(clusters) )
no_noise = np.sum(np.array(clusters) == -1, axis=0)

print(f'Estimated no. of clusters: {no_clusters}')
print(f'Estimated no. of noise points: {no_noise}')
    
# plot the data
plt.scatter(dihedral_data[::10,0], dihedral_data[::10,1], c=clusters, marker="o", picker=True)
```


## 5. Conclusion
<a id='conclusion'></a>

<div class="alert alert-info">
    <b>Key points:</b>   

- Clustering is a form of unsupervised learning
- Unsupervised learning algorithms do not need training   
- K-means is a popular clustering algorithm
- K-means struggles where one cluster exists within another, such as concentric circles
- DBSCAN and Spectral clustering are two other technique which can overcome some of the limitations of k-means 
- Spectral clustering is much slower than k-means
- Different clustering methods can also be applied to real world data such as alanine-dipeptide
- As well as providing machine learning algorithms scikit learn also has functions to make example data. 
</div>

---